
! Este cuadernillo no se puede ejecutar, prueba el código en un.py

# **Human-in-the-Loop en AutoGen**

El enfoque **Human-in-the-Loop** (HITL) en AutoGen permite la interacción directa entre un usuario humano y un equipo de agentes en tiempo real. Esto es útil para: 

✅ **Dar feedback en tiempo real** mientras el equipo está en ejecución.  
✅ **Modificar la conversación antes de continuar** en una nueva ejecución.  
✅ **Combinar IA y supervisión humana** para mejorar respuestas.

---

EJEMPLO EN hip.py

## **1️⃣ Métodos para Proporcionar Feedback**

Existen **dos formas principales** de interactuar con un equipo de agentes y proporcionar feedback:

|**Método**|**Descripción**|**Uso recomendado**|
|---|---|---|
|**Durante la ejecución**|Se usa un `UserProxyAgent` para que el usuario pueda intervenir en tiempo real.|Para interacciones **inmediatas** (aprobaciones, selecciones rápidas).|
|**Después de la ejecución**|Se proporciona feedback en la siguiente llamada a `run()`.|Para ajustes en una **sesión persistente**.|

---

## **2️⃣ Proporcionar Feedback Durante una Ejecución**

Se usa el **`UserProxyAgent`**, un agente especial que actúa como **intermediario entre el usuario y el equipo**.

📌 **¿Cómo funciona?**

1. El equipo ejecuta su tarea.
2. En cierto momento, **se detiene y espera la intervención del usuario**.
3. Una vez el usuario proporciona feedback, la ejecución continúa.

💡 **Ejemplo: Usar `UserProxyAgent` en un equipo de RoundRobinGroupChat**

In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])


In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Crear los agentes
assistant = AssistantAgent("assistant", model_client=model_client)

# Agente que permite intervención humana a través de input()
user_proxy = UserProxyAgent("user_proxy", input_func=input)

# Condición de terminación: el equipo se detendrá si el usuario dice "APPROVE"
termination = TextMentionTermination("APPROVE")

# Crear el equipo con el asistente y el usuario proxy
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Ejecutar el equipo en streaming
await Console(team.run_stream(task="Write a 4-line poem about the ocean."))


---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Waves dance under the sun's warm embrace,  
Whispers of secrets in a vast, blue space.  
Tides that awaken the shores with a song,  
In the heart of the ocean, where dreams belong.


## **3️⃣ Proporcionar Feedback Después de la Ejecución**

Este método se usa cuando el usuario o una aplicación deben **ajustar la tarea antes de volver a ejecutarla**.

📌 **¿Cómo funciona?**

1. El equipo ejecuta su tarea y **se detiene**.
2. El usuario revisa los resultados y proporciona **nuevas instrucciones**.
3. El equipo **se reanuda con la nueva entrada**.

💡 **Ejemplo: Usar `max_turns` para forzar una pausa tras cada respuesta**

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Crear los agentes
assistant = AssistantAgent("assistant", model_client=model_client)

# Crear el equipo con un límite de 1 turno por ejecución
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Write a 4-line poem about the ocean."

while True:
    # Ejecutar la conversación
    await Console(team.run_stream(task=task))
    
    # Pedir feedback al usuario
    task = input("Enter your feedback (type 'exit' to leave): ")
    
    # Salir del bucle si el usuario lo decide
    if task.lower().strip() == "exit":
        break


## **4️⃣ Uso de HandoffTermination**

Algunas tareas requieren que el agente **transfiera la ejecución al usuario** cuando no puede completar la tarea.

💡 **Ejemplo: Agente que transfiere la tarea al usuario**

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Crear cliente del modelo
model_client = OpenAIChatCompletionClient(model="gpt-4o")

# Agente que transfiere la tarea al usuario si no puede responder
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="If you cannot complete the task, transfer to user. Otherwise, respond with 'TERMINATE'.",
)

# Definir condiciones de terminación
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")

# Crear el equipo con ambas condiciones
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=handoff_termination | text_termination)

# Ejecutar el equipo
await Console(lazy_agent_team.run_stream(task="What is the weather in New York?"))


---------- user ----------
What is the weather in New York?
---------- lazy_assistant ----------
[FunctionCall(id='call_XtLGpV0vQTJbvhR7DYbrraSh', arguments='{}', name='transfer_to_user')]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', call_id='call_XtLGpV0vQTJbvhR7DYbrraSh')]
---------- lazy_assistant ----------
Transfer to user.


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=66, completion_tokens=12), content=[FunctionCall(id='call_XtLGpV0vQTJbvhR7DYbrraSh', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, content=[FunctionExecutionResult(content='Transfer to user.', call_id='call_XtLGpV0vQTJbvhR7DYbrraSh')], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=None, target='user', content='Transfer to user.', context=[], type='HandoffMessage')], stop_reason='Handoff to user from lazy_assistant detected.')